In [1]:
import json

In [2]:
%run lib.ipynb

---
# Def

In [3]:
def parseGameById( gameFileId ):
        
    # gameFileId = '8f36d70b-a463-452e-9f02-65dd5d37d2c9'

    with open( f"./output/games/game_{gameFileId}.json") as f:
        data = json.load(f)

    _gameObj = DictObj(data)
        
    home_counter = []
    away_counter = []

    home_reference = _gameObj.home.reference
    away_reference = _gameObj.away.reference

    report = {
        'teams':{
            'home':{
                'reference': _gameObj.home.reference,
                'name': _gameObj.home.name,
                'market': _gameObj.home.market,
            },
            'away':{
                'reference': _gameObj.away.reference,
                'name': _gameObj.away.name,
                'market': _gameObj.away.market,
            }
        },
        'scoring':{
            'total':{
                'home': _gameObj.home.points,
                'away': _gameObj.away.points,
            },
            'periods':[]
        },
        'shots':{}

    }


    for period in _gameObj.periods:

        scoringObj = {
            'home': period.scoring.home.points,
            'away': period.scoring.away.points,
            'period_sequence':  period.sequence,
            'period_number':  period.number,
            'period_type':  period.type
        }
        report['scoring']['periods'].append(scoringObj)

        for event in period.events:

            if( event.event_type == "shotmissed" or  
                event.event_type == "shotsaved" or 
                event.event_type == "goal" ):

                current_len = 0

                if(event.attribution.reference == home_reference):
                    current_len = len(home_counter) + 1
                else:
                    current_len = len(away_counter) + 1


                secondsPeriodByType = seconds_in_period

                if( period.type == 'overtime'):
                     secondsPeriodByType = seconds_in_overtime

                timeInPeriod = secondsPeriodByType - timeToSeconds(event.clock)

                timeInGame = 0

                if( period.sequence == 1):
                    timeInGame = timeInPeriod
                elif ( period.sequence == 2 or period.sequence == 3):
                     timeInGame = ((period.number - 1) * secondsPeriodByType ) + timeInPeriod
                else:
                    # overtime
                    timeInGame = (seconds_in_period * 3 )  +  ((period.sequence - 4) * seconds_in_overtime ) + timeInPeriod

                obj =  {
                    'clock_in_period':  event.clock,
                    'time_game_seconds': timeInGame,
                    'time_period_seconds':  timeInPeriod ,
                    'shot_type':  event.event_type,
                    'shot_counter_onclock': current_len,
                    'period_number':  period.number,
                    'period_type':  period.type,
                    'period_sequence':  period.sequence,
                    'attribution': 'home' if event.attribution.reference == _gameObj.home.reference else 'away'
                }

                if(event.attribution.reference == home_reference):
                    home_counter.append(obj)
                else:
                    away_counter.append(obj)


    report['shots'] = {
        'home':home_counter,
        'away':away_counter,
    }
    
    return report

In [4]:
def getRoundGroupData( groupItem, round ):
    
    teams = groupItem.teams
    team_a = teams[0]
    team_b = teams[1]
    
    _groupReport = {
        'team_a': team_a,
        'team_b': team_b,
        'round': round,
        'games_reports':[]
    }
        
    num_games = len(groupItem.games)
    
    for gameItem in groupItem.games:
        game_score = gameItem.score.split(':')
        
        score_a = game_score[0]
        score_b = game_score[1]

        game_id = gameItem.id
        
        reportData = parseGameById( game_id )
        
        _reportObj = {
            'schedule_data':  gameItem.__dict__,
            'report': reportData
        }
        
        _groupReport['games_reports'].append(_reportObj)
        
    file_name = f"./output/reports/widget-shots-counter/widget-shots-counter_{team_a}-{team_b}.json"
    with open(file_name, "w") as outfile:
        json.dump(_groupReport, outfile, indent=2)
    
    return _groupReport

--- 
# Init

In [5]:
with open( "./data/playoff-schedule.json") as f:
    schedule_data = json.load(f)

schedule = DictObj(schedule_data)

In [8]:
# Change this and run:  0,1,2
roundIdx = 2 

roundItem = schedule.rounds[roundIdx]
round = roundItem.round

for groupItem in roundItem.groups:
    group_report = getRoundGroupData(groupItem, round)